In [49]:
import pandas as pd
import numpy as np
import warnings

import statsmodels.api as sm

from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

In [50]:
def preprocess_data(data, features, imputer):
    data[features] = imputer.transform(data[features])
    return data

In [51]:
def train_model_statsmodels(X, y):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit(cov_type='HC0')
    return model

In [52]:
def create_k_folds(df, fold_n=5):
    folds = {}
    fold_size = len(df) // fold_n
    for i in range(fold_n):
        start = i * fold_size
        if i == fold_n - 1:  # In the last fold, include all remaining data
            end = len(df)
        else:
            end = start + fold_size
        folds[i] = df[start:end]
    return folds

In [53]:
def find_index(lst, target):
    for i, number in enumerate(lst):
        if number == target:
            return i
    return None

In [55]:
def train_final_model(folds_X, folds_y):
    cv_amount = len(folds_X)
    models, rmse_scores, predictions = {}, {}, {}
    rmse_total = 0
    
    for i in range(cv_amount):
        test_X, test_y = folds_X[i], folds_y[i]
        train_X = np.concatenate([folds_X[n] for n in range(len(folds_X)) if n != i])
        train_y = np.concatenate([folds_y[n] for n in range(len(folds_y)) if n != i])

        models['model_{}'.format(i)] = train_model_statsmodels(train_X, train_y)
        test_X = sm.add_constant(test_X, has_constant='add')
        y_pred = models['model_{}'.format(i)].predict(test_X)
        predictions[i] = y_pred
        
        rmse_scores[i] = mean_squared_error(np.log1p(test_y), np.log1p(y_pred), squared=False)
        rmse_total += rmse_scores[i]

    # Final model and the final model's score
    rmse_average = rmse_total / cv_amount
    rmse_list = []
    for i in range(len(rmse_scores)):
        rmse_list.append(rmse_scores[i])
    max_rmse = max(rmse_list)
    min_rmse = min(rmse_list)
    index_max_rmse = find_index(rmse_list, max_rmse)
    index_min_rmse = find_index(rmse_list, min_rmse)
    final_model = models['model_{}'.format(index_max_rmse)]
    display(final_model.summary())
    
    # Residual analysis
    #residuals_cv = np.expm1(folds_y[index_min_r2]) - predictions[index_min_r2]
    #plot_residuals(residuals_cv, predictions[index_min_r2])
    #normality(residuals_cv)
    #print(residuals_cv.sort_values())
    print("-----------------------------------------------------------------------------")
    print("All RMSE score:  ", rmse_scores)
    print("Max RMSE score:  ", rmse_scores[index_max_rmse], index_max_rmse)
    print("Min RMSE score:  ", rmse_scores[index_min_rmse], index_min_rmse)
    print("Average RMSE score:  ", rmse_average)
    return final_model, rmse_average

In [56]:
def cyclic_features(df, df_):
    df['date'] = pd.to_datetime(df['date'])
    
    days_in_year = 365.25
    df['day_of_year'] = df['date'].dt.dayofyear
    df['year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / days_in_year)
    df['year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / days_in_year)
    
    days_in_month = 30.437
    df['day_of_month'] = df['date'].dt.day
    df['month_sin'] = np.sin(2 * np.pi * df['day_of_month'] / days_in_month)
    df['month_cos'] = np.cos(2 * np.pi * df['day_of_month'] / days_in_month)
    
    days_in_week = 7
    df['day_of_week'] = df['date'].dt.dayofweek
    df['week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / days_in_week)
    df['week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / days_in_week)
    

    df_['date'] = pd.to_datetime(df_['date'])
    
    days_in_year = 365.25
    df_['day_of_year'] = df_['date'].dt.dayofyear
    df_['year_sin'] = np.sin(2 * np.pi * df_['day_of_year'] / days_in_year)
    df_['year_cos'] = np.cos(2 * np.pi * df_['day_of_year'] / days_in_year)
    
    days_in_month = 30.437
    df_['day_of_month'] = df_['date'].dt.day
    df_['month_sin'] = np.sin(2 * np.pi * df_['day_of_month'] / days_in_month)
    df_['month_cos'] = np.cos(2 * np.pi * df_['day_of_month'] / days_in_month)
    
    days_in_week = 7
    df_['day_of_week'] = df_['date'].dt.dayofweek
    df_['week_sin'] = np.sin(2 * np.pi * df_['day_of_week'] / days_in_week)
    df_['week_cos'] = np.cos(2 * np.pi * df_['day_of_week'] / days_in_week)
    return df, df_

In [57]:
train_path = 'train.csv'
test_path = 'test.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train, test = cyclic_features(train, test)

selected_features = ['store_nbr', 'onpromotion', 'year_sin', 'year_cos', 'month_sin', 'month_cos', 'week_sin', 'week_cos']

In [63]:
imputer = SimpleImputer(strategy='constant', fill_value=0)
train[selected_features] = imputer.fit_transform(train[selected_features])

test = preprocess_data(test, selected_features, imputer)

y = train['sales']
X = train[selected_features]
X_test = test[selected_features]

folds_X = create_k_folds(X, fold_n=5)
folds_y = create_k_folds(y, fold_n=5)

In [64]:
final_model, rmse_average = train_final_model(folds_X, folds_y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     3666.
Date:                Wed, 13 Mar 2024   Prob (F-statistic):               0.00
Time:                        18:21:35   Log-Likelihood:            -5.6044e+06
No. Observations:             2400711   AIC:                         1.121e+07
Df Residuals:                 2400702   BIC:                         1.121e+07
Df Model:                           8                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9260      0.004    800.395      0.000       2.919       2.933
x1            -0.0009      0.000     -8.801      0.000      -0.001      -0.001
x2             0.0704      0.000    144.119      0.000       0.069       0.071
x3            -0.1031      0.002    -44.001      0.000      -0.108      -0.098
x4             0.0130      0.002      5.790      0.000       0.009       0.017
x5             0.0158      0.002      6.954      0.000       0.011       0.020
x6             0.0124      0.002      5.434      0.000       0.008       0.017
x7            -0.1703      0.002    -73.669      0.000      -0.175      -0.166
x8             0.0841      0.002     36.833      0.000       0.080       0.089
==============================================================================
Omnibus:                    22418.773   Durbin-Watson:                   1.713
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27897.034
Skew:                           0.162   Prob(JB):                         0.00
Kurtosis:                       3.417   Cond. No.                         65.1
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

-----------------------------------------------------------------------------
All RMSE score:   {0: 1.0333124044248962, 1: 0.9333799851315836, 2: 0.8517975042854175, 3: 0.7017428416010756, 4: 0.6530481500544084}
Max RMSE score:   1.0333124044248962 0
Min RMSE score:   0.6530481500544084 4
Average RMSE score:   0.8346561770994763


In [65]:
model = train_model_statsmodels(X, y)

X = sm.add_constant(X)
y_pred = model.predict(X)
rmse_score = mean_squared_error(np.log1p(y), np.log1p(y_pred), squared=False)
print("RMSE", rmse_score)

RMSE 0.8408271708845323


In [66]:
X_test = sm.add_constant(X_test)
predicted_log_sales = model.predict(X_test)
predicted_sales = np.expm1(predicted_log_sales)

In [67]:
predicted_sales_df = pd.DataFrame({
    'id': test['id'],
    'sales': predicted_sales
})
predicted_sales_df.to_csv('predicted_store_sales_statsmodels.csv', index=False)